In [ ]:
import numpy as np
from scipy.linalg import expm
import matplotlib.pyplot as plt
import imageio.v2 as imageio
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
from qutip import *

In [ ]:
N = 30
All_spins_up = np.array([1+0j] + [0+0j]*N)

S_plus = np.zeros((N + 1, N + 1), dtype = complex)
for n in range(N): S_plus[n,n+1] = np.sqrt((n + 1)*(N - n))

S_minus = S_plus.conj().T
S_x = (S_plus + S_minus)/2
S_y = (S_plus - S_minus)/2j
S_z = (S_x @ S_y - S_y @ S_x)/1j

In [ ]:
def rotation_operator(axis, angle):
    axis /= np.linalg.norm(axis)
    return expm(-1j*angle*(axis[0]*S_x + axis[1]*S_y + axis[2]*S_z))

In [ ]:
def evolution(hamiltonian, initial_state, time_step, time_range):
    state = rotation_operator((0,1,0), np.pi/2) @ initial_state
    infinitesimal_evolution = expm(-1j*time_step*hamiltonian)
    result = []
    current_time = 0
    while current_time < time_range:
        result.append(state)
        current_time += time_step
        state = infinitesimal_evolution @ state
        state /= np.linalg.norm(state)
    return result

In [ ]:
time_step = 0.05
time_range = np.pi
time_len = 0
current_time = 0
while current_time < time_range: 
    time_len += 1
    current_time += time_step

In [ ]:
def evol_Wigner_2d(hamiltonian, initial_state):
    frame = 0
    filenames = []
    plots = []
    theta = np.linspace(0, np.pi, 100)
    phi = np.linspace(-np.pi, np.pi, 100)
    for state in evolution(hamiltonian, initial_state, time_step, time_range):
        W, THETA, PHI = spin_wigner(Qobj(state), theta, phi)
        plt.contourf(THETA, PHI, W.real, 100)
        #plt.title("$N=30, H=S_z^2$")
        filename = str(frame) + ".png"
        filenames.append(filename)
        plt.savefig(filename, bbox_inches = "tight", pad_inches = 0, dpi = 216)
        plt.clf()
        plots.append(imageio.imread(filename))
        frame += 1
    imageio.mimsave('wigner2d.gif', plots)
    for filename in set(filenames):
        os.remove(filename)

In [ ]:
def evol_Wigner_3d(hamiltonian, initial_state):
    frame = 0
    filenames = []
    plots = []
    theta = np.linspace(0, np.pi, 300)
    phi = np.linspace(-np.pi, np.pi, 300)
    for state in evolution(hamiltonian, initial_state, time_step, time_range):
        W, THETA, PHI = spin_wigner(Qobj(state), theta, phi)
        colors = W.real
        fig = plt.figure()
        ax = plt.axes(projection ="3d")
        p = ax.scatter(np.sin(THETA)*np.cos(PHI), np.sin(THETA)*np.sin(PHI), np.cos(THETA), c = colors, s = 10)
        # ax.set_title("Wigner function")
        ax.set_xlabel('$S_x$')
        ax.set_ylabel('$S_y$')
        ax.set_zlabel('$S_z$')
        #można obrócić wykres
        #ax.view_init(20, -10)
        filename = str(frame) + ".png"
        filenames.append(filename)
        plt.savefig(filename, bbox_inches = "tight", pad_inches = 0, dpi = 216)
        plt.close()
        plots.append(imageio.imread(filename))
        frame += 1
    imageio.mimsave('wigner3d.gif', plots)
    for filename in set(filenames):
        os.remove(filename)

In [ ]:
def evol_wigner_flat(hamiltonian, initial_state):
    frame = 0
    filenames = []
    plots = []
    theta = np.linspace(0, np.pi, 200)
    phi = np.linspace(-np.pi, np.pi, 200)
    for state in evolution(hamiltonian, initial_state, time_step, time_range):
        W, THETA, PHI = spin_wigner(Qobj(state), theta, phi)
        colors = W.real
        a = 1.340264
        b = -0.081106
        c = 0.000893
        d = 0.003796
        t = np.arcsin(np.sqrt(3)*np.sin(np.pi/2 - THETA)/2)
        xs = (2*PHI*np.cos(t)/np.sqrt(3)/(a+3*b*t**2+7*c*t**6+9*d*t**8))
        ys = (a*t+b*t**3+c*t**7+d*t**9)
        plt.gca().axis("off")
        plt.gca().set_aspect(0.75)
        plt.scatter(xs, ys, c = colors, s = 6)
        # plt.title("Wigner function")
        filename = str(frame) + ".png"
        filenames.append(filename)
        plt.savefig(filename, bbox_inches = "tight", pad_inches = 0, dpi = 216)
        plt.close()

        plots.append(imageio.imread(filename))
        frame += 1
    imageio.mimsave('wigner_flat.gif', plots)
    for filename in set(filenames):
        os.remove(filename)

In [ ]:
evol_Wigner_3d(S_z@S_z, All_spins_up)

In [ ]:
evol_wigner_flat(S_y@S_z+S_z@S_y, All_spins_up)